In [ ]:
import openai
import speech_recognition as sr
import gtts
from dotenv import load_dotenv
import pygame.mixer
import os
from conversation import Conversation

In [ ]:
#load api-keys
load_dotenv()

## Function Setup

In [ ]:
def save_question(question, audio_id):

    # Convert text to speech
    tts = gtts.gTTS(text=question, lang='en')
    path = f"./questions/question_{audio_id}.mp3"
    tts.save(path)
    
    return path

In [ ]:
def play_question(path):
    absolute_path = os.path.abspath(path)
    if os.path.exists(absolute_path):
        try:
            pygame.mixer.init()
            pygame.mixer.music.load(absolute_path)
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                pygame.time.Clock().tick(10)
        except KeyboardInterrupt:
            pygame.mixer.music.stop()
            pygame.mixer.quit()
            print("Playback stopped and resources released.")
        pygame.mixer.quit()
    else:
        print(f"File {absolute_path} does not exist!")


In [ ]:
def get_answer():

    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Please speak something...")
        recognizer.adjust_for_ambient_noise(source, duration=0.2)
        audio_data = recognizer.listen(source)
        print("Recognizing...")

        try:
            # Using Google's speech recognition
            text = recognizer.recognize_google(audio_data)
            print(f"You said: {text}")
            

        except sr.UnknownValueError:
            text = "UnknownValueError"   #REMOVE THIS LATER!!!!!!!!!!!!
            print("I not understand the audio.")
        except sr.RequestError:
            print("API Error.")
    
    return text

In [ ]:
# Initialize OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

def summarize_text(text):
    messages = [
        {"role": "system", "content": "You are a helpful assistant focussing on receiving a good story."},
        {"role": "user", "content": f"Provide a brief summary of the following text and correct any grammar mistakes/wrong words. Dont change the meaning of the text.: '{text}'"}
    ]
    
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
    
    return response.choices[0].message['content'].strip()


In [ ]:
def ask_follow_up_q(questions: str, summary: str):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that asks questions relevant to a story."},
        {"role": "user", "content": f"You are given a summary of a previously asked questions. Ask a relevant follow up Question (that has not been asked before) to get more relevant information about the summary. Use simple Vocabulary and talk like you are in a real conversation: Questions: '{questions}' Summary:'{summary}'"}
    ]
    
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
    
    return response.choices[0].message['content'].strip()

## Pipeline in detail

In [ ]:
audio_id=0
question = "Hey Carl. I always wanted to ask you something. Who was your first big love and how did you two meet?"
path = save_question(question, audio_id)

In [ ]:
play_question(path)

In [ ]:
answer = get_answer()

In [ ]:
answer = "Paris ist the capital off Belgium"
summary = summarize_text(answer)
print(f"Summary: {summary}")

In [ ]:
followUp_q = ask_follow_up_q(question, summary)
print(f"New Question: {followUp_q}")

## Conversation Pipeline

In [ ]:
def question_pipeline(question: str, audio_id: int):
    questions = list()
    questions.append(question)
    
    path = save_question(question, audio_id)
    play_question(path)

    #answer = get_answer()
    answer = "When I was fifteen, I experienced an event that reshaped my perspective on life. My family and I were vacationing in a small coastal village, escaping the city's hustle and bustle. On our third day, while exploring a secluded beach, I found an old, weathered notebook partially buried in the sand. Curiosity piqued, I began reading and realized it was a diary from the 1940s. The entries detailed a woman's love story with a soldier during World War II and the challenges they faced. Their tale of separation, hope, and eventual reunion touched me profoundly, teaching me about the resilience of the human spirit and love's enduring nature. I shared the diary with my family, and it sparked deep conversations about our ancestors and personal challenges. That discovery emphasized the importance of preserving personal histories and cherishing every moment. Today, the diary has a special place in our home, reminding us of the strength we carry within and the bonds that bind us."
    summary = summarize_text(answer)
    print(f"Summary: {summary}")
    
    followUp_q = ask_follow_up_q(questions, summary)
    print(f"New Question: {followUp_q}")
    questions.append(followUp_q)
    
    path = save_question(followUp_q, audio_id+1)
    play_question(path)
    
    answer2 = get_answer()
    summary_complete = summarize_text(answer + " "+ answer2)
    return summary_complete

In [ ]:
question_pipeline("Hey Carl. I always wanted to ask you something. Who was your first big love and how did you two meet?",
                  audio_id=0)

When I was fifteen, I experienced an event that reshaped my perspective on life. My family and I were vacationing in a small coastal village, escaping the city's hustle and bustle. On our third day, while exploring a secluded beach, I found an old, weathered notebook partially buried in the sand. Curiosity piqued, I began reading and realized it was a diary from the 1940s. The entries detailed a woman's love story with a soldier during World War II and the challenges they faced. Their tale of separation, hope, and eventual reunion touched me profoundly, teaching me about the resilience of the human spirit and love's enduring nature. I shared the diary with my family, and it sparked deep conversations about our ancestors and personal challenges. That discovery emphasized the importance of preserving personal histories and cherishing every moment. Today, the diary has a special place in our home, reminding us of the strength we carry within and the bonds that bind us.